In [16]:
import algos
import bandit
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
import utils
import pickle

plt.rcParams['figure.figsize'] = [8, 4]
plt.rcParams['figure.dpi'] = 100

N_TASKS = 1000
N_ARMS = 400
OPT_SIZE = 200
HORIZON = 5000
N_EXPERT = None #None: All possible combinations
N_EXPS = 2 #No. of repeat experiments
GAP_THRESHOLD = np.sqrt(N_ARMS*np.log(N_TASKS)/HORIZON)
kwargs = {'task_cache_step': 10,
          'gap_constrain':min(1,GAP_THRESHOLD*1.0005),
          'plot_var': True,
          'is_adversarial': True,
          'timeout': 1.02, # maximum duration for each roll-outs. Unit = minute. -1 = unlimited
          'quiet': True,
         }
if N_EXPERT is not None:
    print(f'N_EXPERT = {N_EXPERT}')
    max_n_expert = len(list(combinations(np.arange(N_ARMS),OPT_SIZE)))
    assert N_EXPERT<=max_n_expert, f"The number of expert ({N_EXPERT}) must be smaller than the maximum combination ({max_n_expert})"
assert N_ARMS<=HORIZON, f"The number of arm ({N_ARMS}) must be smaller than the horizon ({HORIZON})"
print(f'GAP_THRESHOLD = {GAP_THRESHOLD}')

GAP_THRESHOLD = 0.7433844377699677


## Regret vs Number of tasks

In [ ]:
(X, regret_dict, title, xlabel, ylabel) = utils.task_exp(N_EXPS, N_TASKS, N_ARMS, HORIZON, OPT_SIZE, N_EXPERT, **kwargs)
pickle.dump(regret_dict, open("results/tasks.p", "wb"))
regret_dict = pickle.load(open("results/tasks.p", "rb"))
pass

In [ ]:
# # Plot from loaded cache file
# X = np.arange(N_TASKS)
# gap = kwargs['gap_constrain']
# regret_dict = pickle.load(open("results/tasks_cache.p", "rb"))
# n_experts=10
# title = f'Regret: {N_ARMS} arms, horizon {HORIZON}, {n_experts} experts, gap = {gap:.3f} and subset size {OPT_SIZE}'
# xlabel, ylabel = 'Number of tasks', 'Average Regret per task'
# indices = np.arange(0, X.shape[0], kwargs['task_cache_step']).astype(int)
# utils.plot(X[indices], regret_dict, title, xlabel, ylabel, kwargs['plot_var'])

## Regret vs Horizon

In [ ]:
horizon_list = np.arange(50, 310, 50)
(X_h, regret_dict_h, title, xlabel, ylabel) = \
utils.horizon_exp(N_EXPS, N_TASKS, N_ARMS, OPT_SIZE, N_EXPERT, 
                  horizon_list = horizon_list, **kwargs)
pickle.dump(regret_dict_h, open("results/horizon.p", "wb"))
regret_dict_h = pickle.load(open("results/horizon.p", "rb"))
pass

In [ ]:
# # Plot from loaded cache file
# X_h = np.arange(50, 310, 50)
# n_experts=10
# regret_dict_h = pickle.load(open("results/horizon.p", "rb"))
# title = f'Regret: {N_ARMS} arms, {N_TASKS} tasks, {n_experts} experts, gap cond. satisfied and subset size {OPT_SIZE}'
# xlabel, ylabel = 'Horizon', 'Average Regret per Step'
# utils.plot(X_h, regret_dict_h, title, xlabel, ylabel, kwargs['plot_var'])

## Regret vs # of arm

In [ ]:
n_arms_list = np.arange(3,8,1)
(X_b, regret_dict_b, title, xlabel, ylabel) = \
utils.arms_exp(N_EXPS, N_TASKS, HORIZON, OPT_SIZE, N_EXPERT, n_arms_list, **kwargs)
pickle.dump(regret_dict_b, open("results/arms.p", "wb"))
regret_dict_h = pickle.load(open("results/arms.p", "rb"))
pass

In [ ]:
# # Plot from loaded cache file
# title = f'Regret: Horizon {HORIZON}, {N_TASKS} tasks, all experts, gap cond. satisfied and subset size {OPT_SIZE}'
# xlabel, ylabel = 'Number of Arms', 'Regret'
# X_b = np.arange(3,8,1)
# regret_dict_b = pickle.load(open("results/arms.p", "rb"))
# utils.plot(X_b, regret_dict_b, title, xlabel, ylabel, kwargs['plot_var'])

## Regret vs subset size

In [ ]:
(X_e, regret_dict_e, title, xlabel, ylabel) = utils.subset_exp(N_EXPS, 
                                  N_TASKS, N_ARMS, HORIZON, N_EXPERT, 
                                  opt_size_list = np.arange(1,N_ARMS+1,1), **kwargs)
pickle.dump(regret_dict_e, open("results/subset.p", "wb"))
regret_dict_h = pickle.load(open("results/subset.p", "rb"))
pass

In [ ]:
# # Plot from loaded cache file
# title=f'Regret: {N_ARMS} arms, Horizon {HORIZON}, {N_TASKS} tasks, gap = {gap:.3f} and all experts'
# xlabel, ylabel = 'subset size', 'Regret'
# X_e = np.arange(1,N_ARMS+1,1)
# regret_dict_e = pickle.load(open("results/subset.p", "rb"))
# utils.plot(X_e, regret_dict_e, title, xlabel, ylabel, kwargs['plot_var'])